In [ ]:
from dotenv import load_dotenv
import os

from langchain_gigachat.chat_models import GigaChat

load_dotenv()

GIGACHAT_API_KEY = os.getenv("GIGACHAT_CREDENTIALS")
if not GIGACHAT_API_KEY:
    raise ValueError("❌ GIGACHAT_CREDENTIALS не найден в .env")

langchain_llm = GigaChat(
    credentials=GIGACHAT_API_KEY,
    verify_ssl_certs=False,
    model="GigaChat-Max",   
    temperature=0.4,
    max_tokens=2048,
)

print("✅ GigaChat LangChain LLM инициализирован")


✅ GigaChat LangChain LLM инициализирован


In [55]:
from types import SimpleNamespace
from typing import List, Dict

import autogen
from autogen import AssistantAgent, UserProxyAgent, LLMConfig


class GigaChatLangChainClient:
    """
    Минимальный клиент для AG2, который внутри использует langchain_llm (GigaChat).
    """

    def __init__(self, config, **kwargs):
        self.model_name = config.get("model", "giga-chat-langchain")

    def create(self, params):
        """
        AG2 передаёт сюда messages и параметры.
        Мы вызываем langchain_llm и возвращаем объект с .choices[...].message.content.
        """
        messages: List[Dict[str, str]] = params["messages"]
        prompt = "\n".join(f"{m['role']}: {m['content']}" for m in messages)

        text = langchain_llm.predict(prompt)

        response = SimpleNamespace()
        response.model = self.model_name
        response.choices = []

        choice = SimpleNamespace()
        msg = SimpleNamespace()
        msg.content = text
        msg.function_call = None
        choice.message = msg

        response.choices.append(choice)
        return response

    def message_retrieval(self, response):
        return [choice.message.content for choice in response.choices]

    def cost(self, response) -> float:
        return 0.0

    @staticmethod
    def get_usage(response):
        return {}


print("✅ GigaChatLangChainClient определён")


✅ GigaChatLangChainClient определён


In [44]:
DATA_DESCRIPTION = """
У тебя есть таблица транзакций с такими колонками:

- transaction_id — идентификатор транзакции (целое число)
- transaction_date — дата (строка/дата)
- transaction_qty — количество товара в транзакции (число)
- store_id — идентификатор магазина
- product_id — идентификатор товара
- unit_price — цена за единицу (может быть с копейками)
- product_category — категория товара (например, Coffee, Tea, Bakery)
- product_type — тип товара (например, Brewed coffee, Chai tea, Hot chocolate)
- product_detail — конкретный SKU / название позиции (например, Ethiopia Rg, Latte Rg)

Данные лежат в файле 'transactions.xlsx' в текущей директории.
"""
print("✅ Описание данных подготовлено")


✅ Описание данных подготовлено


In [45]:
llm_config = LLMConfig({
    "model": "giga-chat-langchain",
    "model_client_cls": "GigaChatLangChainClient",
})


In [46]:
planner_system_message = f"""
Ты аналитик по данным в продуктовой команде.

{DATA_DESCRIPTION}

Твоя задача — по текстовому запросу пользователя:

1. Формализовать требования к выборке/отчёту.
2. Ясно перечислить, какие метрики/поля должны быть в финальном pandas DataFrame.
3. Описать формулы вычисления метрик (словами или псевдокодом).
4. Описать группировки / фильтры / сортировки.
5. Если важны ограничения по датам, обязательно зафиксируй период.

Формат ответа:
- Кратко переформулируй задачу.
- Явно перечисли шаги подготовки данных.
- Явно опиши структуру целевого DataFrame в виде списка столбцов с пояснениями, например:

result_df:
  - store_id: идентификатор магазина
  - total_revenue: суммарная выручка = sum(transaction_qty * unit_price)
  - ...

Не пиши Python-код. Пиши только постановку задачи и формальное описание шага за шагом.
"""

planner = AssistantAgent(
    name="planner",
    system_message=planner_system_message,
    llm_config=llm_config,
)


[autogen.oai.client: 11-29 12:58:43] {929} INFO - Detected custom model client in config: GigaChatLangChainClient, model client can not be used until register_model_client is called.


In [47]:
coder_system_message = f"""
Ты Python-разработчик, который пишет код для анализа данных с помощью pandas.

{DATA_DESCRIPTION}

Ты получаешь на вход формализованное ТЗ от аналитика (описание шагов и целевого DataFrame).
Твоя задача — написать ПОЛНЫЙ код на Python, который:

- Импортирует необходимые библиотеки (как минимум: import pandas as pd).
- Загружает данные из файла 'transactions.xlsx' (лист по умолчанию).
- Выполняет все фильтры, агрегации и расчёт метрик, которые описал аналитик.
- Создаёт финальный pandas DataFrame с указанной структурой и кладёт его в переменную result_df.
- В конце печатает:
    - result_df.shape
    - первые несколько строк result_df (например, result_df.head())

Формат ответа:
- ТОЛЬКО один блок с кодом в формате markdown:
```python
# твой код
БЕЗ дополнительного текста до или после блока.
Код должен быть запускаемым 'как есть'.
"""

coder = AssistantAgent(
name="coder",
system_message=coder_system_message,
llm_config=llm_config,
)

[autogen.oai.client: 11-29 12:58:45] {929} INFO - Detected custom model client in config: GigaChatLangChainClient, model client can not be used until register_model_client is called.


In [48]:
### 4.3. Исполнитель (executor) — запускает код

executor = UserProxyAgent(
    name="executor",
    human_input_mode="NEVER",
    code_execution_config={
        "work_dir": ".",      # где запускать код
        "use_docker": False,  # можно True, если хочешь изолировать окружение
    },
)

In [49]:
result_system_message = f"""
Ты итоговый аналитик-контролёр.

{DATA_DESCRIPTION}

Ты получаешь на вход:
- исходный запрос пользователя,
- формализованное ТЗ от аналитика,
- код на Python, который был выполнен,
- текстовый вывод исполнения (stdout/ошибки).

Твоя задача — проанализировать ситуацию и вернуть ОДИН из двух вариантов JSON:

1) Если код успешно отработал и из получившихся данных можно сделать выводы:
{{
  "status": "REPORT",
  "report": "здесь связный аналитический отчёт на русском языке по результатам (что получилось)"
}}

2) Если:
   - при выполнении кода была ошибка (например, есть 'Traceback' или 'Error' в выводе),
   - или result_df пустой/подозрительно маленький,
   - или по данным нельзя сделать осмысленных выводов,
то верни:
{{
  "status": "NEED_FIX",
  "feedback": "что именно не так и что нужно скорректировать в коде или ТЗ"
}}

Требования:
- Ответ ДОЛЖЕН быть строго валидным JSON-объектом без лишнего текста и без ``` вокруг.
- Поле status должно быть только 'REPORT' или 'NEED_FIX'.
"""

result_agent = AssistantAgent(
    name="result_agent",
    system_message=result_system_message,
    llm_config=llm_config,
)


[autogen.oai.client: 11-29 12:58:50] {929} INFO - Detected custom model client in config: GigaChatLangChainClient, model client can not be used until register_model_client is called.


In [50]:
planner.register_model_client(model_client_cls=GigaChatLangChainClient)
coder.register_model_client(model_client_cls=GigaChatLangChainClient)
result_agent.register_model_client(model_client_cls=GigaChatLangChainClient)

print("✅ Агенты planner, coder, executor, result_agent настроены")


✅ Агенты planner, coder, executor, result_agent настроены


In [51]:
import json
from IPython.display import display, Markdown


def run_full_pipeline(user_query: str, max_code_iterations: int = 2):
    """
    Полный пайплайн:
    user_query -> planner -> coder -> executor -> result_agent
    Если result_agent говорит NEED_FIX, даём кодеру ещё попытку (до max_code_iterations раз).
    """

    print("🔹 Запрос пользователя:")
    print(user_query)

    # Шаг 1. Аналитик формализует ТЗ
    print("\n🧠 Ответ аналитика (planner):\n")
    planner_reply = planner.generate_reply(
        messages=[{"role": "user", "content": user_query}]
    )
    print(planner_reply)

    last_code = None
    last_exec_output = None
    iteration = 0

    while iteration < max_code_iterations:
        iteration += 1
        print(f"\n💻 Итерация кодера #{iteration}\n")

        # Шаг 2. Кодер пишет код
        coder_prompt = (
            "Ниже приведено формализованное ТЗ от аналитика.\n"
            "На его основе напиши код согласно своим инструкциям.\n\n"
            "=== ТЗ аналитика ===\n"
            f"{planner_reply}\n"
            "=== Конец ТЗ ===\n"
        )

        coder_reply = coder.generate_reply(
            messages=[{"role": "user", "content": coder_prompt}]
        )
        last_code = coder_reply

        print("📦 Сгенерированный код:\n")
        print(coder_reply)

        # Шаг 3. Исполнитель запускает код (UserProxyAgent)
        print("\n▶️ Запускаем код через executor...\n")

        exec_output = executor.generate_reply(
            messages=[{"role": "user", "content": coder_reply}]
        )
        last_exec_output = exec_output

        print("🖨️ Вывод исполнения (stdout/ошибки):\n")
        print(exec_output)

        # Шаг 4. Результирующий агент решает — REPORT или NEED_FIX
        result_prompt = (
            "Вот все материалы для анализа:\n\n"
            f"Запрос пользователя:\n{user_query}\n\n"
            f"Формализованное ТЗ от аналитика:\n{planner_reply}\n\n"
            "Код, который был выполнен:\n"
            f"{coder_reply}\n\n"
            "Вывод исполнения (stdout/ошибки):\n"
            f"{exec_output}\n\n"
            "Сформируй JSON-ответ согласно твоей инструкции."
        )


        result_reply = result_agent.generate_reply(
            messages=[{"role": "user", "content": result_prompt}]
        )

        print("\n📬 Ответ result_agent (сырые данные):\n")
        print(result_reply)

        # Пробуем распарсить JSON
        try:
            result_json = json.loads(result_reply)
        except Exception as e:
            print("\n⚠️ Не удалось распарсить JSON от result_agent:", e)
            print("Считаем, что нужна правка кода и останавливаемся.")
            break

        status = result_json.get("status")
        if status == "REPORT":
            report_text = result_json.get("report", "")
            print("\n✅ Итоговый отчёт:\n")
            display(Markdown(report_text))
            return {
                "planner_reply": planner_reply,
                "coder_reply": coder_reply,
                "exec_output": exec_output,
                "result_json": result_json,
            }
        elif status == "NEED_FIX":
            feedback = result_json.get("feedback", "")
            print("\n🔁 result_agent считает, что код нужно исправить. Фидбек:\n")
            print(feedback)

            # На следующей итерации можно учесть этот feedback в промпте кодера
            planner_reply += f"\n\nДополнительные замечания по реализации кода:\n{feedback}\n"
            continue
        else:
            print("\n⚠️ Неожиданный статус от result_agent:", status)
            break

    print("\n⏹ Цикл генерации кода завершён без успешного отчёта.")
    return {
        "planner_reply": planner_reply,
        "coder_reply": last_code,
        "exec_output": last_exec_output,
        "result_json": None,
    }


In [67]:
user_query = """
Хочу узнать информацию за январь 2023 года:
- выручку (общую),
- средний чек (общий),
- самые продаваемые напитки в категориях Coffee и Tea,
"""

result = run_full_pipeline(user_query, max_code_iterations=2)


🔹 Запрос пользователя:

Хочу узнать информацию за январь 2023 года:
- выручку (общую),
- средний чек (общий),
- самые продаваемые напитки в категориях Coffee и Tea,


🧠 Ответ аналитика (planner):

### Переформулировка задачи:
Необходимо получить отчет по транзакциям за январь 2023 года, включающий следующие показатели:
- общую выручку компании;
- общий средний чек;
- список наиболее популярных товаров в категориях Coffee и Tea.

---

### Шаги подготовки данных:
1. Загрузить данные из файла `transactions.xlsx`.
2. Преобразовать поле `transaction_date` в корректный формат даты.
3. Отфильтровать строки, соответствующие январю 2023 года (`transaction_date >= '2023-01-01' and transaction_date < '2023-02-01'`).
4. Рассчитать необходимые агрегаты и выбрать нужные поля.

---

### Структура итогового DataFrame:
```python
result_df:
  - category: категория продукта ('Coffee', 'Tea')
  - top_product: самый популярный товар в категории
  - sales_count: количество продаж самого популярного товара
 

По итогам анализа транзакционных данных за январь 2023 года получены следующие результаты:

- Общая выручка составила **81677.74** рублей.
- Средний чек составил примерно **4.72** рубля.
- Самые популярные товары в категориях:
  * В категории "Coffee": продукт ID **27**, продано **551** единица.
  * В категории "Tea": продукт ID **54**, продано **546** единиц.

In [33]:
import pandas as pd

# Чтение данных
df = pd.read_excel('transactions.xlsx')

# Фильтрация дат
# mask = (pd.to_datetime(df['transaction_date']) >= '2023-01-01') & (pd.to_datetime(df['transaction_date']) < '2023-01-08')
# df = df[mask]

# Фильтрация категорий
# categories_of_interest = ['Coffee', 'Tea']
# df = df.query("product_category.isin(@categories_of_interest)")

# Группировка и агрегирование
result_df = (df.groupby(['store_id', 'product_category'])
             .agg(total_sales=('unit_price', 'sum'),
                  total_units=('transaction_qty', 'sum'),
                  avg_price_per_unit=('unit_price', 'mean'),
                  unique_product_count=('product_id', 'nunique'))
             .reset_index())

In [64]:
df

,transaction_id,transaction_date,transaction_qty,store_id,product_id,unit_price,product_category,product_type,product_detail
0,1,2023-01-01,2,5,32,3.00,Coffee,Gourmet brewed coffee,Ethiopia Rg
1,2,2023-01-01,2,5,57,3.10,Tea,Brewed Chai tea,Spicy Eye Opener Chai Lg
2,3,2023-01-01,2,5,59,4.50,Drinking Chocolate,Hot chocolate,Dark chocolate Lg
3,4,2023-01-01,1,5,22,2.00,Coffee,Drip coffee,Our Old Time Diner Blend Sm
4,5,2023-01-01,2,5,57,3.10,Tea,Brewed Chai tea,Spicy Eye Opener Chai Lg
...,...,...,...,...,...,...,...,...,...
3808,3838,2023-01-07,1,8,43,3.00,Tea,Brewed herbal tea,Lemon Grass Lg
3809,3839,2023-01-07,2,8,58,3.50,Drinking Chocolate,Hot chocolate,Dark chocolate Rg
3810,3840,2023-01-07,2,8,24,3.00,Coffee,Drip coffee,Our Old Time Diner Blend Lg
3811,3841,2023-01-07,1,8,58,3.50,Drinking Chocolate,Hot chocolate,Dark chocolate Rg


In [21]:
result_df.head()

,store_id,product_category,total_sales,total_units,avg_price_per_unit,unique_product_count
0,3,Bakery,25844.75,7496,3.545720,11
1,3,Branded,5457.00,279,19.559140,3
2,3,Coffee,60560.30,29655,3.024235,20
3,3,Coffee beans,10219.20,502,20.356972,10
4,3,Drinking Chocolate,17822.75,6351,4.144826,4
